---

# Exercício - Árvores de Decisões

---

**Autor**

> Vitor Eduardo de Souza Costa (13902723)

**Referências**
> Brucce N. dos Santos e Solange O. Rezende. [Prática de Árvores de Decisão](https://edisciplinas.usp.**br**/mod/resource/view.php?id=5293825). Mai. de 2024.

In [ ]:
# @title Importando as bibliotecas necessárias

import pandas as pd, graphviz
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier, export_graphviz
from sklearn.model_selection import train_test_split

---
## Árvore de decisão
---

### Criar dataset do Cadastro dos Pacientes

In [ ]:
# @title Criando dataset

%%writefile cadastro_pacientes_train.tsv
Nome;Febre;Enjôo;Manchas;Dores;Diagnóstico
João;sim;sim;pequenas;sim;doente
Pedro;não;não;grandes;não;saudável
Maria;sim;sim;pequenas;não;saudável
José;sim;não;grandes;sim;doente
Ana;sim;não;pequenas;sim;saudável
Leila;não;não;grandes;sim;doente

Overwriting cadastro_pacientes_train.tsv


In [ ]:
# @title Lendo dataset
# Indexa pelo nome, busca predizer a coluna "Diagnóstico"
dataset = pd.read_csv('cadastro_pacientes_train.tsv', index_col='Nome', sep=';')

dataset

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
João,sim,sim,pequenas,sim,doente
Pedro,não,não,grandes,não,saudável
Maria,sim,sim,pequenas,não,saudável
José,sim,não,grandes,sim,doente
Ana,sim,não,pequenas,sim,saudável
Leila,não,não,grandes,sim,doente


### Limpeza e Tratamento dos dados

> No exercício proposto para a predição do diagnóstico de pacientes dado um conjunto de dados fornecido, não há muito o que ser tratado, portanto algumas etapas a seguir poderiam ser descartadas. Porém, visando aplicar e fixar o aprendizado obtido na aula prática, iremos replicar o que é realizado mesmo que desnecessário.

In [ ]:
# @title Quantidade de valores nulos por coluna

dataset.isnull().sum(axis=0)


Febre          0
Enjôo          0
Manchas        0
Dores          0
Diagnóstico    0
dtype: int64

In [ ]:
# @title Remove exemplos duplicados

dataset.drop_duplicates(inplace=True)

In [ ]:
# @title Imprime os valores únicos de cada coluna

for col in dataset.columns:
  print(col, dataset[col].unique(), sep='\n\t')

Febre
	['sim' 'não']
Enjôo
	['sim' 'não']
Manchas
	['pequenas' 'grandes']
Dores
	['sim' 'não']
Diagnóstico
	['doente' 'saudável']


In [ ]:
# @title transformando dados categóricos em numéricos (distância unitária com dados não ordinais)

dataset.Febre.replace({'sim': 1, 'não': 0}, inplace=True)
dataset.Enjôo.replace({'sim': 1, 'não': 0}, inplace=True)
dataset.Manchas.replace({'grandes': 1, 'pequenas': 0}, inplace=True)
dataset.Dores.replace({'sim': 1, 'não': 0}, inplace=True)
dataset.Diagnóstico.replace({'saudável': 1, 'doente': 0}, inplace=True)

dataset

,Febre,Enjôo,Manchas,Dores,Diagnóstico
Nome,,,,,
João,1,1,0,1,0
Pedro,0,0,1,0,1
Maria,1,1,0,0,1
José,1,0,1,1,0
Ana,1,0,0,1,1
Leila,0,0,1,1,0


In [ ]:
# @title Preencher dados faltantes com zero
## Não é a melhor prática e nem mesmo é necessário em nosso dataset, apenas para fins didáticos

dataset.fillna(0, inplace=True)

In [ ]:
# @title Separa a classe do dataset dos demais atributos
dataset

labels = dataset.Diagnóstico

# Remove atributos que não serão utilizados na classificação, incluindo a classe
dataframe = dataset.drop(['Diagnóstico'], inplace=False, axis=1)

dataframe

,Febre,Enjôo,Manchas,Dores
Nome,,,,
João,1,1,0,1
Pedro,0,0,1,0
Maria,1,1,0,0
José,1,0,1,1
Ana,1,0,0,1
Leila,0,0,1,1


### Construção/Treinamento

In [ ]:
  # @title Determinando árvore de decisões

  ## Instancia o classificador, definindo o critério como entropia
  tree = DecisionTreeClassifier(criterion="entropy")

  ## Treinar a árvore de decisões usando todos os dados
  tree.fit(dataframe, labels)

DecisionTreeClassifier(criterion='entropy')

In [ ]:
# @title Exibindo a árvore de decisões

## plotando a árvore em pdf para melhor visualização
labels_name = ['Doente', 'Saudável']
graph_date = export_graphviz(tree, feature_names=dataframe.columns, class_names=labels_name, filled=True)
graph = graphviz.Source(graph_date)
graph.render('diagrama_arvore')

'diagrama_arvore.pdf'

### Classificação/Avaliação

In [ ]:
# @title Separando o conjunto de dados em treino e teste
treino, teste, labels_treino, labels_teste = train_test_split(dataframe, labels, train_size=0.75,test_size=0.25)

## Construindo árvore de decisão com os dados de treino
tree_avaliacao = DecisionTreeClassifier(criterion="entropy")
tree_avaliacao.fit(treino, labels_treino)

## Construindo árvore de decisão com os dados de teste
predito_tree_avaliacao = tree_avaliacao.predict(teste)

#### Verificando valores preditos em comparação com reais

In [ ]:
temp = teste.copy()
temp['predito'] = predito_tree_avaliacao
temp['real'] = labels_teste

In [ ]:
temp['predito']

Nome
João    0
Ana     0
Name: predito, dtype: int64

In [ ]:
temp['real']

Nome
João    0
Ana     1
Name: real, dtype: int64

In [ ]:
quantidade_acertos = (temp['predito'] == temp['real']).sum()

quantidade_acertos

1

In [ ]:
acuracia = (temp['predito'] == temp['real']).mean()

acuracia

0.5

> Observemos que como o conjunto de dados é mínimo, tanto o conjunto de treino quanto teste fica extremamente pequeno, portanto, a precisão para acertar é extremamente prejudicada tal qual a acurácia, estes valores obtidos são ruins e extremamente sensíveis dependendo de quais dados são selecionados para o treino, variando conforme a execução do código.

---
## Predição para novos casos
---

> Podemos definir os novos casos estabelecidos no exercício proposto e utilizar as árvores de decisão definidas anteriormente. Podemos aplicar tanto a árvore de decisões determinada usando todo o conjunto de dados quanto a que possuiu uma separação para treino e teste (apresentando péssima acurácia pela falta de dados), comparar e discutir os resultados.

### Definindo novo dataframe com dados a serem previstos

In [ ]:
# @title Determinando dataframe para ser previsto

#Criando arquivo
%%writefile novos_pacientes_predict.tsv
Nome;Febre;Enjôo;Manchas;Dores
Luis;não;não;pequenas;sim
Laura;sim;sim;grandes;sim

Overwriting novos_pacientes_predict.tsv


In [ ]:
# @title Lendo novo dataframe
dataframe_predict = pd.read_csv('novos_pacientes_predict.tsv', index_col='Nome', sep=';')

#Atribuindo valor para variáveis categóricas
dataframe_predict.Febre.replace({'sim': 1, 'não': 0}, inplace=True)
dataframe_predict.Enjôo.replace({'sim': 1, 'não': 0}, inplace=True)
dataframe_predict.Manchas.replace({'grandes': 1, 'pequenas': 0}, inplace=True)
dataframe_predict.Dores.replace({'sim': 1, 'não': 0}, inplace=True)

### Predição utilizando árvore treinada com todos os casos

In [ ]:
predito_all = tree.predict(dataframe_predict)
result_all = dataframe_predict.copy()
result_all['Diagnóstico'] = predito_all

## Retornando interpretação numérica dos diagnósticos para categórica
result_all.Diagnóstico.replace({1: 'Saudável', 0: 'Doente'}, inplace=True)

result_all['Diagnóstico']

Nome
Luis     Saudável
Laura      Doente
Name: Diagnóstico, dtype: object

### Predição utilizando árvore treinada e testada

In [ ]:
predito_tested = tree_avaliacao.predict(dataframe_predict)
result_tested = dataframe_predict.copy()
result_tested['Diagnóstico'] = predito_tested

## Retornando interpretação numérica dos diagnósticos para categórica
result_tested.Diagnóstico.replace({1: 'Saudável', 0: 'Doente'}, inplace=True)

result_tested['Diagnóstico']

Nome
Luis     Doente
Laura    Doente
Name: Diagnóstico, dtype: object